In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date,time,datetime
from scipy.stats import norm
import matplotlib.pyplot as plt
data1 = pd.read_csv('/Users/xwx/Desktop/徐丸絮/研究生学习/发表论文/data/usefuldata.csv',index_col=0)
data1.head()

,date,type,direction,time_dept,time_arr,label,hour,time
index,,,,,,,,
2017-01-30 10:00:00,2017/1/30,平日,1,10:18:31,10:55:11,2200,10,2017/1/30 10
2017-01-02 06:00:00,2017/1/2,平日,1,6:41:09,7:20:10,2341,6,2017/1/2 6
2017-01-02 06:00:00,2017/1/2,平日,1,6:48:27,7:34:29,2762,6,2017/1/2 6
2017-01-02 06:00:00,2017/1/2,平日,1,6:58:12,7:41:01,2569,6,2017/1/2 6
2017-01-02 07:00:00,2017/1/2,平日,1,7:04:51,7:50:23,2732,7,2017/1/2 7


In [ ]:
'''
#data2加入了天气数据（用不用就看自己了）
wea = pd.read_csv('/Users/xwx/Desktop/徐丸絮/研究生学习/发表论文/data//wea.csv')
#wea['datetime'] = pd.to_datetime(wea['datetime'])
wea.rename(columns={'datetime':'date','time':'hour'},inplace = True)
#wea.head() #4307
data2 = pd.merge(data1,wea,on=['date','hour'],how='left')
feature_wea = ['tem','dew_point','humidity','pressure','wind_dr','wind_sp','condition']
data2.head()
'''

In [19]:
from sklearn import preprocessing
#提取需要的特征
data=data1[['date','direction','label']]
#对date,direction编码
values=data.values
encoder = preprocessing.LabelEncoder()#LabelEncoder是用来对分类型特征值进行编码，即对不连续的数值或文本进行编码
values[:,0] = encoder.fit_transform(values[:,0])
values[:,1] = encoder.fit_transform(values[:,1])
#归一化
min_max_scaler = preprocessing.MinMaxScaler()
values_minmax = min_max_scaler.fit_transform(values)

/Users/xwx/anaconda3/envs/WZTensorflow_Py37/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [22]:
#构造数据
from sklearn import preprocessing
#构造时间序列
'''
col=list()
col.append(data1.label.shift(1))
col.append(data1.label)
agg = pd.concat(col, axis=1)
agg.columns=['input_seq','output_seq']
agg.dropna(inplace=True)
'''
# convert series to supervised learning

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1] #变量的个数
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1): #n_in代表输入往下退的次数，相当于把后n个挤掉了
        cols.append(df.shift(i))#将数据向下移动i格但是索引值不变
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]#存放变量名
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):#把后n+1个作为输出
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    #然后将输入输出合并 Var1（t-1）那一列代表的就是前t-1时刻var1的时间序列之后作为输入
    #var1(t)那一列代表后t时刻（t，t+1...t+n）的var1的时间序列，是之后的输出序列
    agg = pd.concat(cols, axis=1)
    agg.columns = names#为序列命名
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)#是否过滤空数据
    return agg

#定义LSTM的结构
timesteps=1
features=3

agg=series_to_supervised(values_minmax,timesteps,1)#构造成前timesteps个小时作为输入，下一个时刻作为输出

#预测值只有var3（t），去掉不需要的列
agg.drop(agg.columns[[3,4]],axis=1,inplace=True)
#temp=agg[agg.columns[[5,6,11]]]
agg.head()


,var1(t-1),var2(t-1),var3(t-1),var3(t)
1,0.388889,0.0,0.035676,0.073784
2,0.166667,0.0,0.073784,0.187568
3,0.166667,0.0,0.187568,0.135405
4,0.166667,0.0,0.135405,0.179459
5,0.166667,0.0,0.179459,0.148919


In [23]:
values=agg.values


#分割训练集与测试集（9000+3000）
X_train=values[0:9000,0:-1]
y_train=values[0:9000,-1]

X_test=values[9000:,0:-1]
y_test=values[9000:,-1]

#构造成LSTM需要的输入格式
X_train=X_train.reshape((9000,timesteps,features))
X_test=X_test.reshape((X_test.shape[0],timesteps,features))

print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(9000, 1, 3) (9000,) (3165, 1, 3) (3165,)


In [26]:
#搭建模型
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"#因为运行的时候好像有什么包冲突所以加一句这个，和模型本身无关

from keras. models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(LSTM(16, input_shape=(timesteps, features)))#input_shape=(timesteps,features)
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(X_train, y_train, epochs=40, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


In [17]:
#模型可视化
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                4736      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 4,769
Trainable params: 4,769
Non-trainable params: 0
_________________________________________________________________


In [25]:
#模型评估
from math import sqrt
from sklearn.metrics import mean_absolute_error,mean_squared_error
# make a prediction
X_test=X_test.reshape((X_test.shape[0],timesteps,features))
yhat = model.predict(X_test)#预测值也是[0,1]之间的，因此要转换回原来的值
X_test = X_test.reshape((X_test.shape[0],timesteps*features))
 
# invert scaling for forecast
inv_yhat = np.concatenate((X_test[:,0:features-1],yhat), axis=1)#按列的方式进行组合
inv_yhat = min_max_scaler.inverse_transform(inv_yhat)#从0~1反变换为真实数据
inv_yhat = inv_yhat[:,-1]
 
# invert scaling for actual
y=y_test.reshape(y_test.shape[0],1)
inv_y = np.concatenate(( X_test[:,0:features-1],y), axis=1)
inv_y = min_max_scaler.inverse_transform(inv_y)
inv_y = inv_y[:,-1]
 
# calculate RMSE、MAE
h=data.label.values[9000:-1]
rmse = sqrt(mean_squared_error(h, inv_yhat))
mae=mean_absolute_error(inv_y, inv_yhat)
mape=100*np.mean(np.abs((inv_y-inv_yhat)/inv_y))
#print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)
print('Test MAPE: %.5f' % mape)

Test MAE: 157.436
Test MAPE: 5.61874


In [ ]:
#画出实际和预测曲线
plt.figure(figsize=(24,8))
plt.plot(list(range(len(inv_y))), inv_y, color='b')
plt.plot(list(range(len(inv_yhat))), inv_yhat, color='r')
plt.show()